In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
from powerlifting import models
from pydantic import ValidationError
from rich import pretty, print
import json

pretty.install()

# HI THERE COWBOY

In [2]:
# practicing scanning through directories to access target files
target_files = {"meet.csv", "entries.csv"}
folders = [x for x in Path('../data').iterdir() if x.is_dir()][:2]

file_paths = []
for folder in folders:
    for datafile in folder.iterdir():
        if datafile.name in target_files:
            file_paths.append(datafile.as_posix())

print(file_paths)

[
    '../data/1069/meet.csv',
    '../data/1069/entries.csv',
    '../data/0987/meet.csv',
    '../data/0987/entries.csv'
]

In [3]:
folders = [x for x in Path('../data').iterdir() if x.is_dir()]
columns = set()
not_found = 0
for folder in folders:
    fpath = folder.joinpath("entries.csv")
    try:
        with open(fpath, "r") as f:
            reader = csv.reader(f)
            for first_line in reader:
                for field in first_line:
                    columns.add(field)
                break
    except FileNotFoundError as e:
        not_found += 1
        continue

print(f"Folders without entries: {not_found}")

Folders without entries: 9

In [4]:
len(columns)

35

In [5]:
sorted(list(columns))

[
    'Age',
    'Bench1Kg',
    'Bench2Kg',
    'Bench3Kg',
    'Bench4Kg',
    'Best3BenchKg',
    'Best3DeadliftKg',
    'Best3SquatKg',
    'BirthDate',
    'BirthYear',
    'BodyweightKg',
    'College/University',
    'Country',
    'CyrillicName',
    'Deadlift1Kg',
    'Deadlift2Kg',
    'Deadlift3Kg',
    'Deadlift4Kg',
    'DeadliftEquipment',
    'Division',
    'Equipment',
    'Event',
    'Name',
    'Place',
    'School',
    'Sex',
    'Squat1Kg',
    'Squat2Kg',
    'Squat3Kg',
    'Squat4Kg',
    'State',
    'Team',
    'Tested',
    'TotalKg',
    'WeightClassKg'
]

In [6]:
columns[:4]

TypeError: 'set' object is not subscriptable

In [ ]:
folders[:2]

[PosixPath('../data/1069'), PosixPath('../data/0987')]

In [ ]:
# getting parts of file paths, not sure if needed...
for fp in file_paths:
    parts = fp.split('/')
    print(parts)

In [ ]:
practice_meet = "../data/2412/meet.csv"

meet = pd.read_csv(practice_meet)
meet = meet.to_dict(orient='records')[0]
meet

{
    'Federation': 'USPA',
    'Date': '2022-02-20',
    'MeetCountry': 'USA',
    'MeetState': 'FL',
    'MeetTown': 'Seminole',
    'MeetName': 'Battle of the Bay VIII'
}

In [7]:
meet_info = models.Meet.New(meet)
meet_info

NameError: name 'meet' is not defined

In [8]:
practice_entries = "../data/2412/entries.csv"

entries = pd.read_csv(practice_entries)
entries = entries.replace(np.nan, None).to_dict(orient='records')
entries[0]

{
    'Place': '1',
    'Name': 'Allison Wilson #1',
    'State': 'FL',
    'WeightClassKg': '67.5',
    'BodyweightKg': 60.7,
    'BirthDate': None,
    'Age': 17,
    'Squat1Kg': 110.0,
    'Squat2Kg': -115.0,
    'Squat3Kg': 115.0,
    'Best3SquatKg': 115.0,
    'Bench1Kg': 75.0,
    'Bench2Kg': 77.5,
    'Bench3Kg': -80.0,
    'Best3BenchKg': 77.5,
    'Deadlift1Kg': 122.5,
    'Deadlift2Kg': -130.0,
    'Deadlift3Kg': -130.0,
    'Best3DeadliftKg': 122.5,
    'TotalKg': 315.0,
    'Deadlift4Kg': None,
    'Event': 'SBD',
    'Equipment': 'Raw',
    'Sex': 'F',
    'Division': 'Juniors 16-17'
}

In [9]:
temp = {
    'Place': '1',
    'Name': 'Allison Wilson #1',
    'State': 'FL',
    'WeightClassKg': 'hi',
    'BodyweightKg': 60.7,
    'BirthDate': None,
    'Age': 17,
    'Squat1Kg': 110.0,
    'Squat2Kg': -115.0,
    'Squat3Kg': 115.0,
    'Best3SquatKg': 115.0,
    'Bench1Kg': 75.0,
    'Bench2Kg': 77.5,
    'Bench3Kg': -80.0,
    'Best3BenchKg': 77.5,
    'Deadlift1Kg': 122.5,
    'Deadlift2Kg': -130.0,
    'Deadlift3Kg': -130.0,
    'Best3DeadliftKg': 122.5,
    'TotalKg': 315.0,
    'Deadlift4Kg': None,
    'Event': 'SBD',
    'Equipment': 'Raw',
    'Sex': 'F',
    'Division': 'Juniors 16-17'
}

In [10]:
models.Athlete.New(entries[0])

Athlete(name='Allison Wilson', place='1', state='FL', weight_class=67.5, body_weight=60.7, age=17, best_squat=115.0, best_bench=77.5, best_deadlift=-130.0, total=315.0, event='SBD', equipment='Raw', sex='F', division='Juniors 16-17', squats=Lift(lift1=110.0, lift2=-115.0, lift3=115.0, lift4=None), benches=Lift(lift1=75.0, lift2=77.5, lift3=-80.0, lift4=None), deadlifts=Lift(lift1=None, lift2=None, lift3=None, lift4=None), birth_date=None)

In [11]:
def good_header(fpath: Path) -> bool:
    target_fields = ["Place","Name","State","WeightClassKg","BodyweightKg","BirthDate","Age","Squat1Kg","Squat2Kg","Squat3Kg","Best3SquatKg","Bench1Kg","Bench2Kg","Bench3Kg","Best3BenchKg","Deadlift1Kg","Deadlift2Kg","Deadlift3Kg","Best3DeadliftKg","TotalKg","Bench4Kg","Deadlift4Kg","Event","Equipment","Sex","Division"]
    with open(fpath.as_posix(), "r") as f:
        reader = csv.reader(f)
        header = next(reader)
    return True if  header == target_fields else False


In [12]:
valid_events = 0
athletes = []
for folder in folders:
    for file in folder.iterdir():
        if file.name == "entries.csv" and good_header(file):
            # read file into model
            valid_events += 1
            with open(file.as_posix(), "r") as f:
                reader = csv.DictReader(f)
                for row in reader:
                    athlete = models.Athlete.New(row)
                    athletes.append(athlete)

print(f"Total athletes collected: {len(athletes)} across {valid_events} valid events out of ({round((valid_events / len(folders)) * 100, 2)}%).")
athletes[:2]

Total athletes collected: 2911 across 33 valid events out of (1.4%).

[
    Athlete(name='Ashley Wisner', place='1', state='MD', weight_class=52.0, body_weight=51.5, age=23, best_squat=92.5, best_bench=57.5, best_deadlift=142.5, total=292.5, event='SBD', equipment='Raw', sex='F', division='Juniors 20-23', squats=Lift(lift1=80.0, lift2=85.0, lift3=92.5, lift4=None), benches=Lift(lift1=-55.0, lift2=57.5, lift3=-60.0, lift4=None), deadlifts=Lift(lift1=None, lift2=None, lift3=None, lift4=None), birth_date=None),
    Athlete(name='Michelle Sadler', place='1', state='MD', weight_class=67.5, body_weight=65.4, age=31, best_squat=105.0, best_bench=50.0, best_deadlift=115.0, total=270.0, event='SBD', equipment='Raw', sex='F', division='Open', squats=Lift(lift1=95.0, lift2=100.0, lift3=105.0, lift4=None), benches=Lift(lift1=45.0, lift2=50.0, lift3=-52.5, lift4=None), deadlifts=Lift(lift1=None, lift2=None, lift3=None, lift4=None), birth_date=None)
]

In [13]:
max((athlete.best_bench for athlete in athletes if athlete.best_bench), key=lambda x: x*2)


297.5

In [14]:
sorted((a for a in athletes if a.best_bench), key=lambda x: x.best_bench, reverse=True)[:2]

[
    Athlete(name='Kyle Davis', place='1', state='AZ', weight_class=125.0, body_weight=119.5, age=35, best_squat=285.0, best_bench=297.5, best_deadlift=345.0, total=927.5, event='SBD', equipment='Single-ply', sex='M', division='Open', squats=Lift(lift1=-285.0, lift2=285.0, lift3=-305.0, lift4=None), benches=Lift(lift1=297.5, lift2=-307.5, lift3=None, lift4=None), deadlifts=Lift(lift1=None, lift2=None, lift3=None, lift4=None), birth_date=None),
    Athlete(name='Trey Godbold', place='DQ', state='AL', weight_class=125.0, body_weight=124.1, age=30, best_squat=295.0, best_bench=285.0, best_deadlift=None, total=None, event='SBD', equipment='Raw', sex='M', division='Open', squats=Lift(lift1=280.0, lift2=290.0, lift3=295.0, lift4=None), benches=Lift(lift1=277.5, lift2=285.0, lift3=-288.0, lift4=None), deadlifts=Lift(lift1=None, lift2=None, lift3=None, lift4=None), birth_date=None)
]

In [15]:
final = [a for a in athletes if a.weight_class == 90.0]
len(final)

lst = models.AthleteList(athletes=final)

with open("90kilos_athletes.json", "w") as f:
    f.write(lst.json())

In [7]:
# practicing scanning through directories to access target files
target_files = "meet.csv"
folders = [x for x in Path('../data').iterdir() if x.is_dir()]

file_paths = []
for folder in folders:
    for datafile in folder.iterdir():
        if datafile.name == target_files:
            file_paths.append(datafile.as_posix())

print(file_paths[:4])

[
    '../data/1069/meet.csv',
    '../data/0987/meet.csv',
    '../data/1855/meet.csv',
    '../data/0389/meet.csv'
]

In [8]:
meets = []
for fpath in file_paths:
    with open(fpath, mode="r") as f:
        reader = csv.DictReader(f)
        for event in reader:
            meet = models.Meet.New(event)
            meets.append(meet)

meets[:2]

[
    Meet(federation='USPA', date_=datetime.datetime(2018, 3, 10, 0, 0), country='USA', state='IL', town='Chicago', name='Throwdown at the Compound'),
    Meet(federation='USPA', date_=datetime.datetime(2017, 11, 18, 0, 0), country='USA', state='CA', town='Cotati', name='Sonoma County Open')
]

In [9]:
len(meets)

2344